In [1]:
from utils import *
import time

In [2]:
orders = getAllShopifyOrders()

In [3]:
o=orders[100]

In [4]:
df=pd.DataFrame([o.attributes for o in orders])
df

,id,admin_graphql_api_id,app_id,browser_ip,buyer_accepts_marketing,cancel_reason,cancelled_at,cart_token,checkout_id,checkout_token,...,user_id,billing_address,customer,discount_applications,fulfillments,line_items,refunds,shipping_address,shipping_lines,payment_details
0,3725151600684,gid://shopify/Order/3725151600684,580111,109.246.21.114,True,None,None,2ff2641e24f80d8448589bd029a80e91,2.042036e+13,cb717552e0424b2622ad6b1755113053,...,NaN,billing_address(None),customer(5124872503340),[],[],[line_item(9767295385644)],[],shipping_address(None),[shipping_line(3173021351980)],NaN
1,3723710234668,gid://shopify/Order/3723710234668,580111,177.57.33.194,True,None,None,cb436ca4b1effeadb88766d31a94c72c,2.040661e+13,581a5f1dbbf9c4d2329cde9ce47f402d,...,NaN,billing_address(None),customer(3313802903596),[],[],"[line_item(9764626759724), line_item(976462679...",[],shipping_address(None),[shipping_line(3171802349612)],NaN
2,3723522539564,gid://shopify/Order/3723522539564,580111,200.136.53.132,False,None,None,ff2d2e45d17a541a5e4a277ffb5c34b4,2.040484e+13,22f1180b4976755085a2551576de09b5,...,NaN,billing_address(None),customer(5123183804460),[],[],"[line_item(9764263231532), line_item(976426326...",[],shipping_address(None),[shipping_line(3171646111788)],NaN
3,3722910335020,gid://shopify/Order/3722910335020,580111,177.100.119.80,False,None,None,0e3502e1b1a32657a107600c9767b774,2.039753e+13,3d2d990b9a132900b5b99a588ef8511b,...,NaN,billing_address(None),customer(5122288975916),[],[],"[line_item(9763128246316), line_item(976312827...",[],shipping_address(None),[shipping_line(3171143811116)],NaN
4,3722341777452,gid://shopify/Order/3722341777452,580111,130.132.173.235,False,None,None,98538bb8d83ad65c96f8486e4df71fd6,2.039138e+13,4ea43b70edadf8880e71dbe2bb7c32ed,...,NaN,billing_address(None),customer(4039759167532),[],[],"[line_item(9762085339180), line_item(976208537...",[],shipping_address(None),[shipping_line(3170671591468)],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,2135915823148,gid://shopify/Order/2135915823148,580111,223.190.92.180,True,None,None,fdcb9e2c7f2f802275eba0031f155547,1.255134e+13,d25299e11b8235283003e2b490888957,...,NaN,billing_address(None),customer(3170367569964),[],[fulfillment(1998696513580)],[line_item(4604964044844)],[],shipping_address(None),[shipping_line(1737159704620)],NaN
279,2135884857388,gid://shopify/Order/2135884857388,580111,209.107.188.71,False,None,None,58e4aa9283d24fbfdd9dae68a369f07d,1.255124e+13,3c9a3d3d6b1020915866dc7f098977c0,...,NaN,billing_address(None),customer(3170321367084),[],"[fulfillment(2005009203244), fulfillment(20488...","[line_item(4604904276012), line_item(460490430...",[],shipping_address(None),[shipping_line(1737130868780)],NaN
280,2135852318764,gid://shopify/Order/2135852318764,580111,45.83.89.30,False,None,None,a600c3bd5f6ad02b1402b3a55c050d31,1.255112e+13,5ba1fbfe4eb34b3848fc134535d2ec92,...,NaN,billing_address(None),customer(3170279424044),[],[fulfillment(1998705360940)],[line_item(4604839723052)],[],shipping_address(None),[shipping_line(1737100394540)],NaN
281,2135852318764,gid://shopify/Order/2135852318764,580111,45.83.89.30,False,None,None,a600c3bd5f6ad02b1402b3a55c050d31,1.255112e+13,5ba1fbfe4eb34b3848fc134535d2ec92,...,NaN,billing_address(None),customer(3170279424044),[],[fulfillment(1998705360940)],[line_item(4604839723052)],[],shipping_address(None),[shipping_line(1737100394540)],NaN


In [5]:
[x for x in df.columns if "st" in x]

['customer_locale',
 'financial_status',
 'fulfillment_status',
 'order_status_url',
 'test',
 'total_outstanding',
 'customer']

In [6]:
df["shipping_address"] = df["shipping_address"].apply(lambda x: x.attributes)

In [7]:
for key in df["shipping_address"].iloc[0].keys():
    df["shipping_"+key] = df["shipping_address"].apply(lambda x: x[key])

In [46]:
df["items_shipped"] = [[x.attributes["title"] for x in y] for y in df["line_items"]]

In [48]:
sf = df[["id", "email", "closed_at", "fulfillment_status", "items_shipped"]+[x for x in df.columns if "shipping" in x]].copy()

In [49]:
sf["closed_at"] = pd.to_datetime(sf["closed_at"], utc=True)
sf["closed_at"]

0                           NaT
1                           NaT
2                           NaT
3                           NaT
4                           NaT
                 ...           
278   2020-05-01 23:06:40+00:00
279   2020-05-26 21:48:06+00:00
280   2020-05-01 23:12:45+00:00
281   2020-05-01 23:12:45+00:00
282                         NaT
Name: closed_at, Length: 283, dtype: datetime64[ns, UTC]

In [50]:
sf[sf["fulfillment_status"]=="fulfilled"]

,id,email,closed_at,fulfillment_status,items_shipped,total_shipping_price_set,shipping_address,shipping_lines,shipping_first_name,shipping_address1,...,shipping_province,shipping_country,shipping_last_name,shipping_address2,shipping_company,shipping_latitude,shipping_longitude,shipping_name,shipping_country_code,shipping_province_code
5,3721233465388,a.hegazy@biotec.rwth-aachen.de,2021-05-04 22:12:11+00:00,fulfilled,[One-Step Cloning and Chromosomal Integration ...,total_shipping_price_set(None),"{'first_name': 'Ahmed', 'address1': 'Forckenbe...",[shipping_line(3169689010220)],Ahmed,Forckenbeckstraße 50,...,None,Germany,Hegazy,,DWI Leibniz Institut,50.777274,6.050519,Ahmed Hegazy,DE,None
6,3717445582892,hverdonk@stanford.edu,2021-05-03 19:47:15+00:00,fulfilled,[FreeGenes Cotton Tee],total_shipping_price_set(None),"{'first_name': 'Hannah', 'address1': '1914 Jon...",[shipping_line(3166387765292)],Hannah,1914 Jonathan Ave,...,California,United States,Verdonk,,FreeGenes,37.298027,-121.886167,Hannah Verdonk,US,CA
8,3714284060716,rchurt@caltech.edu,2021-05-04 22:01:21+00:00,fulfilled,[One-Step Cloning and Chromosomal Integration ...,total_shipping_price_set(None),"{'first_name': 'Robert', 'address1': '1200 Eas...",[shipping_line(3163653963820)],Robert,1200 East California Boulevard,...,California,United States,Hurt,MC 210-41,Caltech,34.135667,-118.127240,Robert Hurt,US,CA
9,3709279567916,simons@dwi.rwth-aachen.de,2021-04-26 22:09:11+00:00,fulfilled,"[E. coli Protein Expression Toolkit, Open Repo...",total_shipping_price_set(None),"{'first_name': 'Christian', 'address1': 'Forck...",[shipping_line(3159354310700)],Christian,Forckenbeckstraße 50,...,None,Germany,Simons,,DWI Leibniz Institut,50.777274,6.050519,Christian Simons,DE,None
10,3707102003244,,2021-04-19 22:03:05+00:00,fulfilled,[Open Enzymes],total_shipping_price_set(None),"{'first_name': 'Addgene', 'address1': '490 Ars...",[],Addgene,490 Arsenal Way,...,Massachusetts,United States,Inc.,Suite 100,,42.366605,-71.157703,Addgene Inc.,US,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,2135939547180,,2020-05-13 23:09:21+00:00,fulfilled,"[openEnzyme_1, COVID-19 Mammalian ORFs and Exp...",total_shipping_price_set(None),"{'first_name': 'Khalid', 'address1': '8657 Nar...",[shipping_line(1737181921324)],Khalid,8657 Narragansett Avenue,...,Illinois,United States,Alam,,"Stemloop, Inc.",42.039112,-87.786323,Khalid Alam,US,IL
278,2135915823148,kausik@igib.in,2020-05-01 23:06:40+00:00,fulfilled,[COVID-19 Diagnostic Toolkit Enzymes],total_shipping_price_set(None),"{'first_name': 'Director', 'address1': 'Lab 10...",[shipping_line(1737159704620)],Director,"Lab 104, Mathura Road",...,Delhi,India,CSIR-IGIB,Attention: Kausik Chakraborty,CSIR-IGIB,28.538228,77.288554,Director CSIR-IGIB,IN,DL
279,2135884857388,zach@sound.bio,2020-05-26 21:48:06+00:00,fulfilled,"[openEnzyme_1, COVID-19 Diagnostic Toolkit Enz...",total_shipping_price_set(None),"{'first_name': 'Zach', 'address1': '7732 14th ...",[shipping_line(1737130868780)],Zach,7732 14th Avenue Northeast,...,Washington,United States,Mueller,,SoundBio Lab,47.685955,-122.313056,Zach Mueller,US,WA
280,2135852318764,koeng101@gmail.com,2020-05-01 23:12:45+00:00,fulfilled,[COVID-19 Diagnostic Toolkit Enzymes],total_shipping_price_set(None),"{'first_name': 'Keoni', 'address1': '9082 Pion...",[shipping_line(1737100394540)],Keoni,9082 Pioneer Dr,...,California,United States,Gandall,,DIY,33.674421,-117.969816,Keoni Gandall,US,CA


In [51]:
sf

,id,email,closed_at,fulfillment_status,items_shipped,total_shipping_price_set,shipping_address,shipping_lines,shipping_first_name,shipping_address1,...,shipping_province,shipping_country,shipping_last_name,shipping_address2,shipping_company,shipping_latitude,shipping_longitude,shipping_name,shipping_country_code,shipping_province_code
0,3725151600684,yw14n20@soton.ac.uk,NaT,None,[Open Reporters],total_shipping_price_set(None),"{'first_name': 'Yi', 'address1': 'Life Science...",[shipping_line(3173021351980)],Yi,"Life Sciences Building 85,",...,England,United Kingdom,Wu,"Highfield Campus, Southampton SO17 1BJ",university of southampton,50.936319,-1.395312,Yi Wu,GB,ENG
1,3723710234668,robson.tramontina@gmail.com,NaT,None,"[Mushroom Genetic Engineering Toolkit v0.01, O...",total_shipping_price_set(None),"{'first_name': 'robson', 'address1': 'Estrada ...",[shipping_line(3171802349612)],robson,Estrada Giuseppina Vianelli di Napolli,...,São Paulo,Brazil,tramontina,1455,ECRA Biotec,-22.811540,-47.048018,robson tramontina,BR,SP
2,3723522539564,vitoriavarollo@usp.br,NaT,None,"[Open Plasmids, One-Step Cloning and Chromosom...",total_shipping_price_set(None),"{'first_name': 'Vitória', 'address1': 'Avenida...",[shipping_line(3171646111788)],Vitória,"Avenida Professor Lineu Prestes, 2242",...,São Paulo,Brazil,Varollo,CIETEC - Sala 226 - 2° andar,University of São Paulo,-23.566798,-46.727655,Vitória Varollo,BR,SP
3,3722910335020,ueira@ufu.br,NaT,None,"[Open Reporters, Molecular Diagnostics Toolkit]",total_shipping_price_set(None),"{'first_name': 'Carlos', 'address1': 'Acre Str...",[shipping_line(3171143811116)],Carlos,Acre Street Bloco 2E room 230,...,Minas Gerais,Brazil,Ueira-Vieira,,Federal University of Uberlândia - UFU,NaN,NaN,Carlos Ueira-Vieira,BR,MG
4,3722341777452,kyle.mohler@yale.edu,NaT,None,"[Glowing Plant, One-Step Cloning and Chromosom...",total_shipping_price_set(None),"{'first_name': 'Kyle', 'address1': '300 Heffer...",[shipping_line(3170671591468)],Kyle,300 Heffernan Drive,...,Connecticut,United States,Mohler,"Rinehart Lab, ISTC Rm 313",Yale University - West Campus,41.256467,-72.990800,Kyle Mohler,US,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,2135915823148,kausik@igib.in,2020-05-01 23:06:40+00:00,fulfilled,[COVID-19 Diagnostic Toolkit Enzymes],total_shipping_price_set(None),"{'first_name': 'Director', 'address1': 'Lab 10...",[shipping_line(1737159704620)],Director,"Lab 104, Mathura Road",...,Delhi,India,CSIR-IGIB,Attention: Kausik Chakraborty,CSIR-IGIB,28.538228,77.288554,Director CSIR-IGIB,IN,DL
279,2135884857388,zach@sound.bio,2020-05-26 21:48:06+00:00,fulfilled,"[openEnzyme_1, COVID-19 Diagnostic Toolkit Enz...",total_shipping_price_set(None),"{'first_name': 'Zach', 'address1': '7732 14th ...",[shipping_line(1737130868780)],Zach,7732 14th Avenue Northeast,...,Washington,United States,Mueller,,SoundBio Lab,47.685955,-122.313056,Zach Mueller,US,WA
280,2135852318764,koeng101@gmail.com,2020-05-01 23:12:45+00:00,fulfilled,[COVID-19 Diagnostic Toolkit Enzymes],total_shipping_price_set(None),"{'first_name': 'Keoni', 'address1': '9082 Pion...",[shipping_line(1737100394540)],Keoni,9082 Pioneer Dr,...,California,United States,Gandall,,DIY,33.674421,-117.969816,Keoni Gandall,US,CA
281,2135852318764,koeng101@gmail.com,2020-05-01 23:12:45+00:00,fulfilled,[COVID-19 Diagnostic Toolkit Enzymes],total_shipping_price_set(None),"{'first_name': 'Keoni', 'address1': '9082 Pion...",[shipping_line(1737100394540)],Keoni,9082 Pioneer Dr,...,California,United States,Gandall,,DIY,33.674421,-117.969816,Keoni Gandall,US,CA


In [52]:
from datetime import datetime
today = datetime.today()
datem = datetime(today.year, today.month, 1)
datem

datetime.datetime(2021, 5, 1, 0, 0)

In [55]:
export = sf[sf["closed_at"].dt.month==(today.month-1)].copy()
export.columns

Index(['id', 'email', 'closed_at', 'fulfillment_status', 'items_shipped',
       'total_shipping_price_set', 'shipping_address', 'shipping_lines',
       'shipping_first_name', 'shipping_address1', 'shipping_phone',
       'shipping_city', 'shipping_zip', 'shipping_province',
       'shipping_country', 'shipping_last_name', 'shipping_address2',
       'shipping_company', 'shipping_latitude', 'shipping_longitude',
       'shipping_name', 'shipping_country_code', 'shipping_province_code'],
      dtype='object')

In [57]:
export = export[[c for c in export.columns if not c in ["total_shipping_price_set", "shipping_lines", "shipping_address"]]].copy()
export.columns

Index(['id', 'email', 'closed_at', 'fulfillment_status', 'items_shipped',
       'shipping_first_name', 'shipping_address1', 'shipping_phone',
       'shipping_city', 'shipping_zip', 'shipping_province',
       'shipping_country', 'shipping_last_name', 'shipping_address2',
       'shipping_company', 'shipping_latitude', 'shipping_longitude',
       'shipping_name', 'shipping_country_code', 'shipping_province_code'],
      dtype='object')

In [62]:
for offset in range(1,5):
    df=pd.DataFrame([o.attributes for o in orders])
    df["shipping_address"] = df["shipping_address"].apply(lambda x: x.attributes)
    for key in df["shipping_address"].iloc[0].keys():
        df["shipping_"+key] = df["shipping_address"].apply(lambda x: x[key])
    df["items_shipped"] = [", ".join([x.attributes["title"] for x in y]) for y in df["line_items"]]
    sf = df[df["fulfillment_status"]=="fulfilled"][["id", "email", "closed_at", "fulfillment_status", "items_shipped"]+[x for x in df.columns if "shipping" in x and x not in ["total_shipping_price_set", "shipping_lines", "shipping_address"]]].copy()
    sf["closed_at"] = pd.to_datetime(sf["closed_at"], utc=True)
    from datetime import datetime
    today = datetime.today()
    date = datetime(today.year, today.month-offset, 1)
    date = date.strftime("%B %Y")
    filename = f"{date}-orders-updated-report.csv"
    export = sf[sf["closed_at"].dt.month==(today.month-offset)].copy()
    with open(filename, "w") as f:
        f.write(export.to_csv())